In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from typing import Dict, Text, Any, Union
from tfx import v1 as tfx
import os
import tempfile
import tensorflow as tf
from transformers import GPT2TokenizerFast
import tensorflow_text as text
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import ml_metadata as mlmd
import constants

import tensorflow as tf
import preprocessing_gpt2
 

2023-03-27 07:38:16.016081: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 07:38:17.232148: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/intel64/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/mic/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/intel64/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/mic/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/intel64/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/mic/lib:/opt/

In [3]:
context = InteractiveContext(constants.PIPELINE_NAME, pipeline_root=constants.ARTIFACT_DIR)
store = mlmd.MetadataStore(context.metadata_connection_config)

In [4]:
path_to_examples = [examples for examples in store.get_artifacts_by_type("Examples") if "CsvExampleGen" in examples.uri][-1].uri
example_gen = tfx.dsl.Importer(
    source_uri=path_to_examples,
    artifact_type=tfx.types.standard_artifacts.Examples,
    reimport=False).with_id('import_examples')
context.run(example_gen)


path_to_schema = [schema for schema in store.get_artifacts_by_type("Schema") if "/SchemaGen/" in schema.uri][-1].uri
schema_gen = tfx.dsl.Importer(
    source_uri=path_to_schema,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False).with_id('import_schema')
context.run(schema_gen)



ExecutionResult(
    component_id: import_schema
    execution_id: 36
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "/home/azureuser/pipelines/artifacts/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema:2023-03-20T13:43:36.583331"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.12.0"
          }
        }
        state: LIVE
        name: "schema:2023-03-20T13:43:36.583331"
        create_time_since_epoch: 1679319816927
        last_update_time_since_epoch: 1679319817040
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [5]:
context.show(schema_gen.outputs["result"])

,Type,Presence,Valency,Domain
Feature name,,,,
'original',BYTES,required,,-
'simplifications',BYTES,required,,-
'text',BYTES,required,,-
'uuid',BYTES,required,,-


In [6]:
text_input = "When it comes to making transformers easy, HuggingFace is the"
processed_data = preprocessing_gpt2.preprocessing_fn( { "text": tf.constant([text_input]) })
print(processed_data)

2023-03-27 07:38:26.917238: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


{'input_ids': <tf.Tensor: shape=(1, 128), dtype=int64, numpy=
array([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256,  2215,   340,  2058,
          284, 

In [7]:
from transformers import TFGPT2LMHeadModel, GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
# encode text
# add left padding
# add mask tokens
input_ids = tokenizer(text_input, return_tensors="tf", truncation=True, max_length=256, padding="max_length", add_special_tokens=False)


In [8]:
input_ids

{'input_ids': <tf.Tensor: shape=(1, 256), dtype=int32, numpy=
array([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 

In [9]:
from transformers import TFAutoModelForCausalLM
gpt_model = TFAutoModelForCausalLM.from_pretrained("gpt2")
# gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")
gpt_model.config.pad_token_id = gpt_model.config.eos_token_id
generated = gpt_model.generate(**input_ids, do_sample=True, max_new_tokens=128)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/transformers/generation/tf_utils.py:603: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [11]:
tokenizer.decode(generated[0], skip_special_tokens=True)

"When it comes to making transformers easy, HuggingFace is the best of the best.\n\nI've tried many brands, from KJW-W's, and I still love making the correct ones for just about everything that you see on your skin. From the skin tone and colors of the eyes to your appearance in everyday life. With the right color combination, you can transform your face into a nice, simple and easy looking smile.\n\nIf you have the time and time to spend with your face, I recommend making a small brush on top of your lashes to help prevent them from clumping.\n\nThe brushes are a great way to remove a few more strands of mascara that"

In [12]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['result'],
    schema=schema_gen.outputs['result'],
    module_file=os.path.abspath("preprocessing_gpt2.py"),
)
context.run(transform)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying constants.py -> build/lib
copying model.py -> build/lib
copying preprocessing.py -> build/lib
copying preprocessing_gpt2.py -> build/lib
copying task.py -> build/lib
installing to /tmp/tmpcud1lrxv
running install
running install_lib
copying build/lib/constants.py -> /tmp/tmpcud1lrxv
copying build/lib/preprocessing.py -> /tmp/tmpcud1lrxv
copying build/lib/preprocessing_gpt2.py -> /tmp/tmpcud1lrxv
copying build/lib/task.py -> /tmp/tmpcud1lrxv
copying build/lib/model.py -> /tmp/tmpcud1lrxv
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing

Instructions for updating:
Use ref() instead.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:tensorflow:Assets written to: /home/azureuser/pipelines/artifacts/Transform/transform_graph/37/.temp_path/tftransform_tmp/c46c2db4514249e183b5696ae7b40fea/assets


INFO:tensorflow:Assets written to: /home/azureuser/pipelines/artifacts/Transform/transform_graph/37/.temp_path/tftransform_tmp/c46c2db4514249e183b5696ae7b40fea/assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 37
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [18]:
import tensorflow_transform as tft

import tensorflow as tf
import constants

def _get_serve_features_signature(model, tf_transform_output):
    """Returns a function that parses a raw inputs and applies TFT."""

    model.tft_layer_inference = tf_transform_output.transform_features_layer()
    tokenizer = preprocessing_gpt2.get_gpt_tokenizer()
    tokenizer.padding_side = "left"
    model.tokenizer = tokenizer

    @tf.function(
        input_signature=(tf.TensorSpec((None,), tf.string, name="text"),)

    )
    def serving(text, context=None):

        text = tf.reshape(text, shape=[-1, 1])

        transformed_features = model.tft_layer_inference({"text": text})
        # return transformed_features
        input_ids = transformed_features["input_ids"]
        attention_mask = transformed_features["attention_mask"]

        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=256,
            return_dict_in_generate=True,
        )

        sentences = model.tokenizer.detokenize(outputs["sequences"])
        return {"outputs" : sentences}

    return serving

def save_model(serving_model_dir, model, tf_transform_output):
    model.save(
        serving_model_dir,
        save_format="tf",
        signatures={
            "serving_default": _get_serve_features_signature(model, tf_transform_output)
        }
    )

In [19]:

from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import tensorflow_transform as tft
output_dir = transform.outputs["transform_graph"]._artifacts[0].uri
tf_transform_output = tft.TFTransformOutput(output_dir)
hf_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [21]:

MODEL_SERVING_DIR = os.path.join(constants.ROOT_DIR, "serving_model/generative_dense_writing")
save_model(MODEL_SERVING_DIR, hf_model, tf_transform_output)

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


INFO:tensorflow:Assets written to: /home/azureuser/serving_model/generative_dense_writing/assets


INFO:tensorflow:Assets written to: /home/azureuser/serving_model/generative_dense_writing/assets


{'outputs': <tf.Tensor: shape=(1,), dtype=string, numpy=
 array([b"<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

In [22]:
text_input = "When it comes to making transformers easy, HuggingFace is the"
loaded_model = tf.saved_model.load(MODEL_SERVING_DIR)
loaded_model.signatures["serving_default"](text=tf.constant([text_input]))

{'outputs': <tf.Tensor: shape=(1,), dtype=string, numpy=
 array([b"<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex